# Let's create Hungry Geese agent by supervised learning!

The purpose of this Notebook is to provide an easy way to participate in this competition, so that many people can enjoy Hungry Geese. Kagglers are familiar with supervised learning and keras is one of the easiest frameworks to use. So, I would like to introduce a simple approach using keras with supervised learning. Of course, if you are aiming for a higher level, you will need reinforcement learning. I'm learning RL through this competition. Let's enjoy Hungry Geese together!

This notebook is based on a lot of public notebooks. In particular, the following ones:  
Most of technical parts including model architecture, data pipeline etc: https://www.kaggle.com/yuricat/smart-geese-trained-by-reinforcement-learning  
Scraping episode files: https://www.kaggle.com/robga/simulations-episode-scraper-match-downloader  

#### import libraries

In [1]:
# import libraries
import pickle
import json
import gc
import bz2
import base64
from glob import glob

import numpy as np
import pandas as pd
from pandas import DataFrame

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, recall_score

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dense, Conv2D, Activation, Lambda, Add, 
                                     BatchNormalization, Input, Lambda)
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l1_l2 ,l1, l2
from tensorflow.keras import mixed_precision

#### Radabelief Optimizer

In [2]:
# https://github.com/ben-arnao/Radabelief
import tensorflow as tf
from tensorflow_addons.utils.types import FloatTensorLike

from typing import Union, Callable, Dict
from typeguard import typechecked
import numpy as np


@tf.keras.utils.register_keras_serializable(package="Addons")
class RadaBelief(tf.keras.optimizers.Optimizer):

    @typechecked
    def __init__(
            self,
            learning_rate: Union[FloatTensorLike, Callable, Dict] = 1e-4,
            beta_1: FloatTensorLike = 0.9,
            beta_2: FloatTensorLike = 0.999,
            epsilon: FloatTensorLike = 1e-12,
            warmup_steps: int = 10000,
            name: str = "Radabelief",
            **kwargs
    ):
        super().__init__(name, **kwargs)

        self._set_hyper("learning_rate", kwargs.get("lr", learning_rate))
        self._set_hyper("beta_1", beta_1)
        self._set_hyper("beta_2", beta_2)
        self._set_hyper("warmup_steps", warmup_steps)
        self.epsilon = epsilon or tf.keras.backend.epsilon()

    def _create_slots(self, var_list):
        for var in var_list:
            self.add_slot(var, "m")
        for var in var_list:
            self.add_slot(var, "v")

    def set_weights(self, weights):
        params = self.weights
        num_vars = int((len(params) - 1) / 2)
        if len(weights) == 3 * num_vars + 1:
            weights = weights[: len(params)]
        super().set_weights(weights)

    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype

        # *current* learn rate of optimizer (changed by ReduceLROnPlateau for example)
        lr_t = self.lr

        # get previous moments
        m = self.get_slot(var, "m")
        v = self.get_slot(var, "v")

        # static hyperparameters of optimizer
        lr_0 = self._get_hyper("learning_rate", var_dtype)
        warmup_steps = self._get_hyper("warmup_steps", var_dtype)
        beta_1 = self._get_hyper("beta_1", var_dtype)
        beta_2 = self._get_hyper("beta_2", var_dtype)

        # smaller epsilon == more bias == more like SGD
        # larger epsilon == more adaptive, potential for large LR difference between variables
        epsilon_t = tf.convert_to_tensor(self.epsilon, var_dtype)

        # current step of optimizer
        local_step = tf.cast(self.iterations + 1, var_dtype)

        # use linearly scaled lr from 0 to initial LR while steps under 'warmup_steps'
        lr = tf.where(
            local_step <= warmup_steps,
            (local_step / warmup_steps) * lr_0,
            lr_t,
        )

        # calculate first moment of gradient (momemtum)
        m_t = m.assign(
            beta_1 * m + (1.0 - beta_1) * grad,
            use_locking=self._use_locking
        )

        # calculate second moment of gradient (RMSprop)
        # use 'tf.square(grad - m_t)' for Adabelief instead of 'tf.square(grad)'
        v_t = v.assign(
            beta_2 * v + (1.0 - beta_2) * tf.square(grad - m_t),
            use_locking=self._use_locking,
        )

        # correct bias (mostly affects initial steps)
        m_corr_t = m_t / (1.0 - tf.pow(beta_1, local_step))
        v_corr_t = v_t / (1.0 - tf.pow(beta_2, local_step))

        # calculate step
        var_t = m_corr_t / (tf.sqrt(v_corr_t) + epsilon_t)

        # apply learn rate
        var_update = var.assign_sub(lr * var_t,
                                    use_locking=self._use_locking)

        updates = [var_update, m_t, v_t]
        return tf.group(*updates)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "learning_rate": self._serialize_hyperparameter("learning_rate"),
                "beta_1": self._serialize_hyperparameter("beta_1"),
                "beta_2": self._serialize_hyperparameter("beta_2"),
                "epsilon": self.epsilon,
                "warmup_steps": self._serialize_hyperparameter("warmup_steps"),
            }
        )
        return config

C:\Users\sohai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


#### Cosine Anealing

In [3]:
import math
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K

class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

 #### Data Generator

In [4]:
# besed on https://github.com/yu4u/mixup-generator
class GeeseDataGenerator(Sequence):
        'Generates data for Keras'
        def __init__(self, X_train, y_train, batch_size=128, shuffle=True, sample_weight=None):
            'Initialization'
            self.batch_size = batch_size
            self.X_train = X_train
            self.y_train = y_train
            self.shuffle = shuffle
            self.on_epoch_end()
            self.sample_weight = sample_weight
    
        def __len__(self):
            'Denotes the number of batches per epoch'
            return int(np.ceil(len(self.X_train) / self.batch_size))
    
        def __getitem__(self, index):
            'Generate one batch of data'
            # Generate indexes of the batch
            indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
            # Generate data
            if self.sample_weight is not None:
                X, y, w = self.__data_generation(indexes)
                return X, y, w
            else:
                X, y = self.__data_generation(indexes)
                return X, y
    
        def on_epoch_end(self):
            'Updates indexes after each epoch'
            self.indexes = np.arange(len(self.X_train))
            if self.shuffle == True:
                np.random.shuffle(self.indexes)
        
        def __data_generation(self, batch_ids):
            #_, h, w, c = self.X_train.shape
        
            X = self.X_train[batch_ids]
            y = self.y_train[batch_ids]
            if self.sample_weight is not None:
                w = self.sample_weight[batch_ids]
            else:
                pass
            
            series_len = len(X)
            
            # vertical flip
            if np.random.uniform(0,1)>0.5:
                X = np.flip(X, axis=2)
                y = y[:,[0,1,3,2]]
                
            # horizontal flip
            if np.random.uniform(0,1)>0.5:
                X = np.flip(X, axis=1)
                y = y[:,[1,0,2,3]] 
            
            if self.sample_weight is not None:
                return X, y, w
            else:
                return X, y

#### Create Data from json

In [5]:
def create_dataset_from_json(filepath, json_object=None, standing=0):
    if json_object is None:
        json_open = open(path, 'r')
        json_load = json.load(json_open)
    else:
        json_load = json_object
        
    try:
        winner_index = np.argmax(np.argsort(json_load['rewards']) == 3-standing)

        obses = []
        X = []
        y = []
        actions = {'NORTH':0, 'SOUTH':1, 'WEST':2, 'EAST':3}

        for i in range(len(json_load['steps'])-1):
            if json_load['steps'][i][winner_index]['status'] == 'ACTIVE':
                y_= json_load['steps'][i+1][winner_index]['action']
                if y_ is not None:
                    step = json_load['steps'][i]
                    step[winner_index]['observation']['geese'] = step[0]['observation']['geese']
                    step[winner_index]['observation']['food'] = step[0]['observation']['food']
                    obses.append(step[winner_index]['observation'])
                    y.append(actions[y_])        

        for j in range(len(obses)):
            X_ = make_input(obses[:j+1])
            X_ = centerize(X_)
            X.append(X_)

        X = np.array(X, dtype=np.float32)#[starting_step:]
        y = np.array(y, dtype=np.uint8)#[starting_step:]

        return X, y
    except:
        return 0, 0

#### Data Pipeline

In [6]:
# based on https://www.kaggle.com/yuricat/smart-geese-trained-by-reinforcement-learning
def centerize(b):
    dy, dx = np.where(b[0])
    centerize_y = (np.arange(0,7)-3+dy[0])%7
    centerize_x = (np.arange(0,11)-5+dx[0])%11
    
    b = b[:, centerize_y,:]
    b = b[:, :,centerize_x]
    
    return b

def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs['index']) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs['index']) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs['index']) % 4, pos] = 1
            
    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[12 + (p - obs['index']) % 4, pos] = 1

    # food
    for pos in obs['food']:
        b[16, pos] = 1
        
    b = b.reshape(-1, 7, 11)
    b = centerize(b) # Where to place the head is arbiterary dicision.

    return b

#### Model Architecture

In [7]:
# based on https://www.kaggle.com/yuricat/smart-geese-trained-by-reinforcement-learning
def TorusConv2D(x, ch, kernel, padding="same", strides=1, weight_decay=2e-3):
    x = Lambda(lambda x: K.tile(x, n=(1,3,3,1)), 
               output_shape=lambda input_shape: (None, 3*input_shape[1], 3*input_shape[2], input_shape[3]))(x)
    
    x = Conv2D(ch, kernel, padding=padding, strides=strides,
                      kernel_regularizer=l2(weight_decay))(x)
    
    x = Lambda(lambda x: x[:,int(x.shape[1]/3):2*int(x.shape[1]/3), int(x.shape[2]/3):2*int(x.shape[2]/3),:], 
               output_shape=lambda input_shape: (None, int(input_shape[1]/3), int(input_shape[2]/3), input_shape[3]))(x)
    return x

def conv_bn_relu(x0, ch, kernel, padding="same", strides=1, weight_decay=2e-3, add=False):
    x = TorusConv2D(x0, ch, kernel, padding=padding, strides=strides,
                      weight_decay=weight_decay)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    if add:
        x = Add()([x0, x])
    return x

def GeeseNet(input_shape=(7, 11, 17), layers=12, filters=32, weight_decay=2e-3, compile=True):
    input = Input(input_shape)
    x = conv_bn_relu(input, filters, 3)
    
    for i in range(layers):
        x = conv_bn_relu(x, filters, 3, add=True)
    
    x = GlobalAveragePooling2D()(x)
    
    output = Dense(4, activation='softmax', kernel_regularizer=l1_l2(l1=0.0005, l2=0.0005))(x)   
    model = Model(input, output)
    if compile:
        model.compile(optimizer=RadaBelief(learning_rate=1e-3, epsilon=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    else:
        pass
    
    return model

In [8]:
# use mixed precision
policy = mixed_precision.Policy('mixed_float16')

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [9]:
# Create data set from json
# Only a few episodes are used to train here. Try it with your own json collection.
paths = [path for path in glob("./flood_fill_3k_data/*.json") if 'info' not in path]

print(len(paths))

X_train = []
y_train = []

for path in tqdm(paths[:int(len(paths))]):
    X, y = create_dataset_from_json(path, standing=0) # use only winners' moves
    if X is not 0:
        X_train.append(X)
        y_train.append(y)
    
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

X_train, unique_index = np.unique(X_train, axis=0, return_index=True)
y_train = y_train[unique_index]

X_train = np.transpose(X_train, [0, 2, 3, 1])
y_train = to_categorical(y_train)

0


<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\sohai\AppData\Local\Temp\ipykernel_28360\458220406.py:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if X is not 0:


0it [00:00, ?it/s]

ValueError: need at least one array to concatenate

In [ ]:
X_train.shape, y_train.shape

In [ ]:
alpha=0.3
batch_size = 32
val_size = 0.1

X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=71)

training_generator = GeeseDataGenerator(X_train_, (1-alpha)*y_train_+0.25*alpha, batch_size=batch_size)
cos = CosineAnnealingScheduler(10, 2e-3*batch_size/32, 0, verbose=1)

model = GeeseNet(input_shape=(7, 11, 17), layers=12, filters=32, weight_decay=1e-7, compile=True)
model.fit(training_generator, validation_data=(X_val, y_val), epochs=11, batch_size=batch_size, callbacks=[cos])
model.save('model.h5')

In [ ]:
# Write weight to the head of the python script
weight_base64 = base64.b64encode(bz2.compress(pickle.dumps(model.get_weights())))
w = "weight= %s"%weight_base64
%store w >submission.py

In [ ]:
%%writefile -a submission.py
import pickle
import bz2
import base64
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Conv2D, Activation, Lambda, Add, BatchNormalization, Input
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l1_l2, l2


# Neural Network for Hungry Geese
def TorusConv2D(x, ch, kernel, padding="same", strides=1, weight_decay=2e-3):
    x = Lambda(lambda x: K.tile(x, n=(1,3,3,1)), 
               output_shape=lambda input_shape: (None, 3*input_shape[1], 3*input_shape[2], input_shape[3]))(x)
    
    x = Conv2D(ch, kernel, padding=padding, strides=strides,
                      kernel_regularizer=l2(weight_decay))(x)
    
    x = Lambda(lambda x: x[:,int(x.shape[1]/3):2*int(x.shape[1]/3), int(x.shape[2]/3):2*int(x.shape[2]/3),:], 
               output_shape=lambda input_shape: (None, int(input_shape[1]/3), int(input_shape[2]/3), input_shape[3]))(x)
    return x

def conv_bn_relu(x0, ch, kernel, padding="same", strides=1, weight_decay=2e-3, add=False):
    x = TorusConv2D(x0, ch, kernel, padding=padding, strides=strides,
                      weight_decay=weight_decay)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    if add:
        x = Add()([x0, x])
    return x

def GeeseNet(input_shape=(7, 11, 17), layers=12, filters=32, weight_decay=2e-3):
    input = Input(input_shape)
    x = conv_bn_relu(input, filters, 3)
    
    for i in range(layers):
        x = conv_bn_relu(x, filters, 3, add=True)
    
    x = GlobalAveragePooling2D()(x)
    
    output = Dense(4, activation='softmax', kernel_regularizer=l1_l2(l1=0.0005, l2=0.0005))(x)   
    model = Model(input, output)
    #model.compile(optimizer=RadaBelief(learning_rate=1e-3, epsilon=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])    
    
    return model

# Input for Neural Network
def centerize(b):
    dy, dx = np.where(b[0])
    centerize_y = (np.arange(0,7)-3+dy[0])%7
    centerize_x = (np.arange(0,11)-5+dx[0])%11
    
    b = b[:, centerize_y,:]
    b = b[:, :,centerize_x]
    
    return b

def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs['index']) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs['index']) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs['index']) % 4, pos] = 1
            
    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[12 + (p - obs['index']) % 4, pos] = 1

    # food
    for pos in obs['food']:
        b[16, pos] = 1
        
    b = b.reshape(-1, 7, 11)
    b = centerize(b) # Where to place the head is arbiterary dicision.

    return b


# Load Keras Model
model = GeeseNet(input_shape=(7, 11, 17), layers=12, filters=32, weight_decay=1e-7)
model.set_weights(pickle.loads(bz2.decompress(base64.b64decode(weight))))

# Main Function of Agent

obses = []

def agent(obs_dict, config_dict):
    obses.append(obs_dict)

    X_test = make_input(obses)
    X_test = np.transpose(X_test, (1,2,0))
    X_test = X_test.reshape(-1,7,11,17) # channel last.
    
    # avoid suicide
    obstacles = X_test[:,:,:,[8,9,10,11,12]].max(axis=3) - X_test[:,:,:,[4,5,6,7]].max(axis=3) # body + opposite_side - my tail
    obstacles = np.array([obstacles[0,2,5], obstacles[0,4,5], obstacles[0,3,4], obstacles[0,3,6]])
    
    y_pred = model.predict(X_test) - obstacles

    
    
    actions = ['NORTH', 'SOUTH', 'WEST', 'EAST']
    return actions[np.argmax(y_pred)]

In [ ]:
from kaggle_environments import make
env = make("hungry_geese", debug=True)
# enemy = '../input/nejugeese/submission_baseline.py'

env.reset()
env.run(['submission.py','submission.py','submission.py','submission.py'])
env.render(mode="ipython", width=600, height=500)